# Child to Parents mappings in StyleGAN2 latent space using the Ridge Regression

In [1]:
from dataset.nok_mean import NokMeanDataset
from dataset.nok import NokDataset
from dataset.nok_aug import NokAugDataset
from utils.stylegan import StyleGAN2
from utils.eval import BaseEvaluator
from utils.viz import image_add_label
import os
import sys
import numpy as np
import torch, torchvision
from PIL import Image

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error


/home/vidp/miniconda3/envs/diploma_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_data(split):
    X = []
    y = []
    for father, mother, child, *_ in NokMeanDataset(split=split):
        input = child.flatten(0)
        output = torch.cat([father.flatten(0), mother.flatten(0)], dim=0)
        X.append(input)
        y.append(output)
    X = torch.stack(X, dim=0)
    y = torch.stack(y, dim=0)
    return X, y

In [3]:
name = "child_to_parent"
output_path = "./.tmp/stylegan2-ada-pytorch/" + name + "001"
while (os.path.exists(output_path)):
  output_path = output_path[:-3] + str(int(output_path[-3:]) + 1).zfill(3)

os.mkdir(output_path)

In [4]:
X_train, y_train = load_data(split="train")
X_test, y_test = load_data(split="test")

Loaded 433 persons with 2676 images.
Average images per person: 6.180138568129331
Max images per person: 37
Min images per person: 1
[13, 12, 6, 1, 4, 2, 6, 9, 8, 2, 4, 7, 1, 6, 3, 3, 5, 8, 12, 4, 3, 6, 20, 4, 10, 3, 7, 3, 1, 4, 5, 9, 5, 8, 2, 4, 4, 3, 3, 2, 11, 2, 7, 14, 8, 10, 2, 1, 2, 9, 11, 5, 1, 1, 5, 2, 5, 5, 8, 1, 1, 5, 5, 3, 1, 3, 7, 23, 14, 15, 16, 5, 4, 1, 3, 9, 5, 6, 5, 1, 2, 5, 16, 4, 2, 1, 5, 2, 6, 24, 5, 15, 3, 9, 13, 2, 5, 6, 9, 7, 4, 8, 4, 7, 5, 11, 4, 9, 7, 6, 6, 7, 8, 5, 7, 5, 18, 10, 6, 8, 1, 5, 5, 9, 14, 6, 6, 1, 11, 14, 2, 7, 1, 2, 4, 4, 17, 5, 1, 6, 6, 4, 2, 3, 4, 5, 1, 2, 4, 4, 9, 3, 6, 2, 3, 5, 8, 6, 7, 14, 6, 5, 18, 9, 5, 3, 13, 3, 12, 5, 14, 5, 5, 1, 5, 9, 2, 9, 2, 6, 10, 8, 10, 12, 9, 6, 19, 10, 7, 37, 9, 6, 1, 1, 4, 24, 7, 3, 3, 4, 6, 13, 5, 8, 4, 5, 4, 2, 24, 5, 19, 6, 5, 13, 5, 3, 7, 8, 11, 4, 12, 7, 8, 7, 14, 15, 6, 8, 4, 5, 3, 5, 6, 8, 11, 6, 2, 2, 10, 4, 4, 3, 1, 1, 1, 1, 2, 1, 2, 5, 1, 16, 9, 8, 4, 4, 1, 9, 3, 5, 7, 8, 3, 2, 6, 4, 1, 5, 2, 4, 5, 6, 4, 

In [5]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape


(torch.Size([207, 9216]),
 torch.Size([207, 18432]),
 torch.Size([20, 9216]),
 torch.Size([20, 18432]))

In [6]:
regressor = Ridge()
regressor.fit(X_train, y_train)

Ridge()

In [7]:
y_train_hat = regressor.predict(X_train)
y_test_hat = regressor.predict(X_test)

In [8]:
mse_train = mean_squared_error(y_train, y_train_hat)
mse_test = mean_squared_error(y_test, y_test_hat)

In [9]:
mse_train, mse_test

(0.006931318833174911, 1.0244137867456073)

In [10]:
images = []
for i in range(y_test.shape[0]):
    images.append(X_test[i].view(18, 512))
    images.append(y_test[i,:18*512].view(18, 512))
    images.append(y_test[i,18*512:].view(18, 512))
    images.append(torch.from_numpy(y_test_hat[i,:18*512]).to(torch.float32).view(18, 512))
    images.append(torch.from_numpy(y_test_hat[i,18*512:]).to(torch.float32).view(18, 512))
images = torch.stack(images, dim=0)

## Visualize and evaluate results

In [11]:
toTensor = torchvision.transforms.PILToTensor()
toPIL = torchvision.transforms.ToPILImage()
eval = BaseEvaluator()
stylegan = StyleGAN2(tmp_path=output_path)

./submodules/stylegan2-ada-pytorch/
./.tmp/stylegan2-ada-pytorch/child_to_parent009
Got StyleGAN2 docker client, building image...
StyleGAN2 Docker image built.


In [12]:
pils = stylegan.generate_from_array(images.detach().cpu().numpy())
pil = toPIL(torchvision.utils.make_grid([toTensor(pil.resize((128,128))) for pil in pils], nrow=5)).convert("RGB")
pil.save(output_path + "/result.png")

Generating 100 images from array...
[INFO] StyleGAN2 - Generating image...
Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"...

Generating images from projected W "./.tmp/stylegan2-ada-pytorch/child_to_parent009/projected_w.npz"

Setting up PyTorch plugin "bias_act_plugin"... Done.

Setting up PyTorch plugin "upfirdn2d_plugin"... Done.



In [13]:
pils_reshaped = np.array([np.array(pil).reshape(1024, 1024, 3) for pil in pils])
pils_reshaped = pils_reshaped.reshape(-1, 5, 1024, 1024, 3)
images_eval_f = pils_reshaped[:, 1]
images_eval_m = pils_reshaped[:, 2]
images_hat_eval_f = pils_reshaped[:, 3]
images_hat_eval_m = pils_reshaped[:, 4]

In [14]:
images_eval_arr_f = [Image.fromarray(i).convert("RGB") for i in images_eval_f]
images_eval_arr_m = [Image.fromarray(i).convert("RGB") for i in images_eval_m]
images_hat_eval_arr_f = [Image.fromarray(i).convert("RGB") for i in images_hat_eval_f]
images_hat_eval_arr_m = [Image.fromarray(i).convert("RGB") for i in images_hat_eval_m]

In [15]:
images_eval_pil_f = toPIL(torchvision.utils.make_grid([toTensor(pil.resize((128, 128))) for pil in images_eval_arr_f], nrow=1)).convert("RGB")
images_eval_pil_f.save(output_path + "/images_eval_f.png")

images_eval_pil_m = toPIL(torchvision.utils.make_grid([toTensor(pil.resize((128, 128))) for pil in images_eval_arr_m], nrow=1)).convert("RGB")
images_eval_pil_m.save(output_path + "/images_eval_m.png")

In [16]:
eval_res_f_fn = eval.evaluate_batch(images_eval_f, images_hat_eval_f, model_name='Facenet512')
images_hat_eval_arr_f_labeled_fn = zip(images_hat_eval_arr_f, eval_res_f_fn)
images_hat_eval_pil_f_fn = toPIL(torchvision.utils.make_grid([toTensor(image_add_label(pil, str(round(label, 3)), 40).resize((256, 256))) for pil, label in images_hat_eval_arr_f_labeled_fn], nrow=1)).convert("RGB")
images_hat_eval_pil_f_fn.save(output_path + "/images_eval_hat_f_fn.png")
eval_res_f_fn

eval_res_m_fn = eval.evaluate_batch(images_eval_m, images_hat_eval_m, model_name='Facenet512')
images_hat_eval_arr_m_labeled_fn = zip(images_hat_eval_arr_m, eval_res_m_fn)
images_hat_eval_pil_m_fn = toPIL(torchvision.utils.make_grid([toTensor(image_add_label(pil, str(round(label, 3)), 40).resize((256, 256))) for pil, label in images_hat_eval_arr_m_labeled_fn], nrow=1)).convert("RGB")
images_hat_eval_pil_m_fn.save(output_path + "/images_eval_hat_m_fn.png")
eval_res_m_fn


ValueError: ('Confirm that opencv is installed on your environment! Expected path ', '/home/vidp/miniconda3/envs/diploma_env/lib/python3.9/site-packages/data/haarcascade_frontalface_default.xml', ' violated.')

In [ ]:
eval_res_f_af = eval.evaluate_batch(images_eval_f, images_hat_eval_f, model_name='ArcFace')
images_hat_eval_arr_f_labeled_af = zip(images_hat_eval_arr_f, eval_res_f_af)
images_hat_eval_pil_f_af = toPIL(torchvision.utils.make_grid([toTensor(image_add_label(pil, str(round(label, 3)), 40).resize((256, 256))) for pil, label in images_hat_eval_arr_f_labeled_af], nrow=1)).convert("RGB")
images_hat_eval_pil_f_af.save(output_path + "/images_eval_hat_f_af.png")
eval_res_f_af

eval_res_m_af = eval.evaluate_batch(images_eval_m, images_hat_eval_m, model_name='ArcFace')
images_hat_eval_arr_m_labeled_af = zip(images_hat_eval_arr_m, eval_res_m_af)
images_hat_eval_pil_m_af = toPIL(torchvision.utils.make_grid([toTensor(image_add_label(pil, str(round(label, 3)), 40).resize((256, 256))) for pil, label in images_hat_eval_arr_m_labeled_af], nrow=1)).convert("RGB")
images_hat_eval_pil_m_af.save(output_path + "/images_eval_hat_m_af.png")
eval_res_m_af


In [ ]:
eval_res_vgg = eval.evaluate_batch(images_eval_f, images_hat_eval_f, model_name='VGG-Face')
images_hat_eval_arr_f_labeled_vgg = zip(images_hat_eval_arr_f, eval_res_vgg)
images_hat_eval_pil_f_vgg = toPIL(torchvision.utils.make_grid([toTensor(image_add_label(pil, str(round(label, 3)), 40).resize((256, 256))) for pil, label in images_hat_eval_arr_f_labeled_vgg], nrow=1)).convert("RGB")
images_hat_eval_pil_f_vgg.save(output_path + "/images_eval_hat_f_vgg.png")
eval_res_vgg

eval_res_vgg = eval.evaluate_batch(images_eval_m, images_hat_eval_m, model_name='VGG-Face')
images_hat_eval_arr_m_labeled_vgg = zip(images_hat_eval_arr_m, eval_res_vgg)
images_hat_eval_pil_m_vgg = toPIL(torchvision.utils.make_grid([toTensor(image_add_label(pil, str(round(label, 3)), 40).resize((256, 256))) for pil, label in images_hat_eval_arr_m_labeled_vgg], nrow=1)).convert("RGB")
images_hat_eval_pil_m_vgg.save(output_path + "/images_eval_hat_m_vgg.png")
eval_res_vgg
